#라이브러리 임포트와 데이터셋 재구성

In [2]:
# 모델 분석 및 시각화 도구
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import xml.etree.ElementTree as ET
import os
from sklearn.metrics import confusion_matrix, accuracy_score

# 모델 설계
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, LeakyReLU, Flatten, GlobalAveragePooling2D
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [3]:
# xml 파일의 라벨을 리턴하는 함수 (필터링용)
def parse_xml(xml_file): #
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # 레이블을 추출하는 부분
    label = None #레이블을 저장할 변수를 미리 정의하고 None으로 초기화 
    for obj in root.iter('object'): #XML 구조에서 object라는 태그를 가진 모든 요소를 반복하며 탐색
        name = obj.find('name').text # 요소 추출: 'object' 태그 안에서 'name' 태그를 찾고, 그 안에 담긴 텍스트 값을 읽어온다.
        if name in ['with_mask', 'mask_weared_incorrect']:  # 유효한 레이블만 처리(조건에 따라)
            label = name #name 값이 ['with_mask', 'mask_weared_incorrect'] 리스트에 포함되어 있다면, label에 저장
            break #탐색을 중단( 필요한 정보를 찾으면 더 이상 읽지 않고 그 자리에서 끝냄)

    return label

변경이 가능한 부분
1. 탐색할 태그 이름 변경('object')
예시: XML 구조에서 <item>을 탐색하려면 'object'=>'item'으로

2. 유효한 레이블 리스트 변경
예) 딸기의 상태를 분류할 경우, 리스트를 ['ripe', 'unripe']로 바꾸면 성숙도 정보를 필터링할 수 있음.

3. 탐색 중단 조건 (break)
모든 유효한 레이블을 찾으려면 break를 제거하고 결과를 리스트에 저장하는 방식으로 바꿀 수 있음.

labels = []
for obj in root.iter('object'):
    name = obj.find('name').text
    if name in ['ripe', 'unripe']:
        labels.append(name)
return labels

break를 제거=> 리스트에 저장하는 코드는 모든 레이블을 한 번에 수집하기 위해 
리스트 저장:
1. 모든 유효한 레이블을 수집하려면 break를 제거해야 XML 파일 끝까지 탐색 가능
2. 수집한 레이블을 편리하게 사용하려면
=>레이블을 리스트에 저장하여 나중에 분석, 카운트, 필터링 등을 쉽게 수행할 수 있음

즉, 리스트 = 수집 가방, 모든 레이블을 한 곳에 담아두는 역할을 해서 이를 통해 레이블을 하니씩 따로 관리하지 
않고, 한꺼번에 활용할 수 있음 

input_dir = '/root/.cache/kagglehub/datasets/andrewmvd/face-mask-detection/versions/1/annotations'
output_dir = 'result/labels'
image_dir = '/root/.cache/kagglehub/datasets/andrewmvd/face-mask-detection/versions/1/images'
# 위의 부분은 YOLO 학습시 이미 정의함

annotations_dir = input_dir
images_dir = image_dir

image_data = [] #image_data: 전처리된 이미지 데이터를 저장할 리스트
labels = [] #labels: 각 이미지에 해당하는 레이블을 저장할 리스트

for xml_file in os.listdir(annotations_dir):
    if xml_file.endswith('.xml'):

        xml_path = os.path.join(annotations_dir, xml_file)
        label = parse_xml(xml_path)

        # 필터링 (without_mask 제외)
        if label is None or label == 'without_mask':
            continue

        image_file = xml_file.replace('.xml', '.png')
        image_path = os.path.join(images_dir, image_file)

        if os.path.exists(image_path):

            image = cv2.imread(image_path)
            image = cv2.resize(image, (128, 128))  # Resize to 128x128

            image_data.append(image)
            labels.append(label)

X = np.array(image_data)
y = np.array(labels)

print("라벨 분포:", dict(zip(*np.unique(y, return_counts=True))))

# yolo 학습용 데이터 전처리 수행 코드 
image_data = []
labels = []
데이터와 레이블 저장 리스트 초기화 : 데이터와 레이블을 담을 빈 바구니를 준비하는 것 => 나중에 탐색하면서 데이터를 하나씩 채움

for xml_file in os.listdir(annotations_dir): #os.listdir: 주석 파일이 있는 디렉토리에서 모든 파일 이름을 가져옴
    if xml_file.endswith('.xml'): #if xml_file.endswith('.xml'): XML 파일만 처리하기 위해 조건을 추가
창고 안에 있는 모든 상자에서, 이름표가  "XML"로 끝나는 상자만 열어보는 것과 비슷

XML 파일 경로 생성 및 레이블 추출
xml_path = os.path.join(annotations_dir, xml_file) #os.path.join: 디렉토리와 파일 이름을 합쳐 파일 경로를 생성
label = parse_xml(xml_path) #parse_xml: XML 파일에서 레이블을 추출하는 함수
상자를 열고 안에 있는 이름표(label)를 확인하는 작업

레이블 필터링
if label is None or label == 'without_mask':
    continue
label이 None이거나, "without_mask"인 경우 데이터를 건너뜀
이 조건을 통해 YOLO 학습에 필요하지 않은 데이터를 필터링. => 유효하지 않은 레이블이 담긴 상자는 버리는 것과 같음

이미지 파일 경로 생성 및 확인 
image_file = xml_file.replace('.xml', '.png') #xml_file.replace('.xml', '.png'): XML 파일 이름에서 .xml을 .png로 바꿔 이미지 이름을 생성
image_path = os.path.join(images_dir, image_file) #이미지 파일이 존재하는지 확인합니다.


if os.path.exists(image_path):
주석 파일에 해당하는 이미지가 실제로 창고에 있는지 확인하는 작업

이미지 읽기 및 크기 조정
image = cv2.imread(image_path) #cv2.imread: OpenCV로 이미지를 읽어오기 
image = cv2.resize(image, (128, 128))  # cv2.resize: 이미지를 128x128 픽셀 크기로 조정

큰 상자를 열어 안에 든 물건(이미지)를 꺼내고, 크기 작은 박스(128x128)로 줄이는 작업

이미지와 레이블 저장
image_data.append(image)
labels.append(label)
전처리된 이미지와 레이블을 각각 준비된 리스트에 추가함

numpy 배열로 변환
X = np.array(image_data)
y = np.array(labels)
image_data와 labels 리스트를 NumPy 배열로 변환 => 리스트에 담긴 물건(데이터)을 기계가 처리하기 쉬운 포장지(NumPy 배열)로 감싸는 작업.

변경 가능한 부분
1. 이미지 크기 조정 => 원하는 크기로 변경 가능 
2. 레이블 필터링 조건 => 원하는 레이블만 포함 
3. 출력 디렉토리 경로 => 경로 변경 가능 




도서관에 책(XML 파일)과 관련 이미지가 있는 상황
책(XML 파일)에는 레이블(책의 주제)이 기록

작업
1. 책을 하나씩 열어보고 필요한 주제만 선택하기
2. 주제에 해당하는 이미지를 가져오기
3. 이미지를 정해진 크기로 줄이고 정리하기
4. 최종적으로 학습에 필요한 데이터(책의 이미지)를 포장하기 


전체 흐름 요약
도서관 위치 설정: XML 파일과 이미지가 저장된 경로를 설정
상자 준비: 데이터를 담을 리스트 만듦
책 탐색: XML 파일을 하나씩 탐색하며 레이블 정보를 추출.
필터링: 유효하지 않거나 관심 없는 데이터를 건너뜀
이미지 처리: 이미지를 읽고 크기를 조정.
데이터 저장: 정리된 데이터를 리스트에 추가.
포장: 데이터를 NumPy 배열로 변환하여 기계 학습에 적합하게 만든다.
통계 출력: 각 레이블의 개수를 출력하여 데이터 분포를 확인

#데이터셋 구성의 핵심
1. 데이터 경로 설정// 주석 파일(XML)과 이미지 파일이 저장된 디렉토리를 지정
2. 레이블 추출 및 필터링//원하는 데이터만 선택하여 노이즈를 제거하는 방법을 보여
3. 이미지 전처리// 모델에 맞게 이미지를 크기 조정하고 배열 형태로 변환
4. 데이터 정리 //최종적으로 모델에 입력할 수 있는 형식으로 데이터를 준비

#데이터셋 구성 시 중요한 개념 적용
유효한 데이터 필터링: 학습에 필요하지 않은 데이터를 제거하는 방식 without_mask 제외).
전처리: 이미지 크기를 조정하거나 데이터를 numpy 배열로 변환해 모델이 이해할 수 있는 형태로 만드는 과정
데이터 분포 확인: 레이블이 골고루 분포되었는지 확인하는 방법

###!!
레이블: 모델이 학습할 목표로 'with_mask', 'without_mask' 같은 것들

골고루 분포라는 것: 각 주제(레이블)별로 비슷한 수의 책(데이터)이 존재해야 함
1. 모델의 학습 균형 유지:
 'with_mask' 레이블이 1000권, 'without_mask' 레이블이 10권이라면, 
 모델은 'with_mask'에 대해 아주 잘 학습하고 'without_mask'에 대해서는 거의 학습하지 못함.
 =>불균형한 학습을 클래스 불균형 (Class Imbalance) 문제라고 함.

 2. 잘못된 예측(bias)
 클래스가 불균형하면 모델이 예측할 때 한쪽으로 편향(bias) 될 가능성이 커짐. 모델이 항상 'with_mask'를 예측한다면 정확도가 높아보일 수 있지만, 
 실제로는 'without_mask'의 예측을 놓치게 되는 문제를 겪게 됨.

 3. 검출 실패 가능성:
 'without_mask' 클래스의 데이터가 적으면 모델은 이 상태를 잘 인식하지 못할 수 있음.
 마스크를 쓰지 않은 사람을 잘못 분류하거나 무시하는 경우가 발생할 수 있음.